# Inicialización y Carga de Datos 

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# 1. Cargar el Dataset
df = pd.read_csv('../csv/seoul_bike_sharing_original.csv')
print("Dataset cargado exitosamente. Las primeras 5 filas:")
print(df.head())

Dataset cargado exitosamente. Las primeras 5 filas:
         Date  Rented Bike Count  Hour  Temperature(°C)  Humidity(%)  \
0  01/12/2017                254     0             -5.2           37   
1  01/12/2017                204     1             -5.5           38   
2  01/12/2017                173     2             -6.0           39   
3  01/12/2017                107     3             -6.2           40   
4  01/12/2017                 78     4             -6.0           36   

   Wind speed (m/s)  Visibility (10m)  Dew point temperature(°C)  \
0               2.2              2000                      -17.6   
1               0.8              2000                      -17.6   
2               1.0              2000                      -17.7   
3               0.9              2000                      -17.6   
4               2.3              2000                      -18.6   

   Solar Radiation (MJ/m2)  Rainfall(mm)  Snowfall (cm) Seasons     Holiday  \
0                      0.0 

In [2]:
# 2. Renombrar y Estandarizar Columnas
# Las columnas del dataset de UCI suelen tener espacios y mayúsculas.
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df.rename(columns={'rented_bike_count': 'demanda'}, inplace=True)

In [3]:
# 3. Imprimimos los primeros 5 con los nombres nuevos
print(df.head())

         date  demanda  hour  temperature(°c)  humidity(%)  wind_speed_(m/s)  \
0  01/12/2017      254     0             -5.2           37               2.2   
1  01/12/2017      204     1             -5.5           38               0.8   
2  01/12/2017      173     2             -6.0           39               1.0   
3  01/12/2017      107     3             -6.2           40               0.9   
4  01/12/2017       78     4             -6.0           36               2.3   

   visibility_(10m)  dew_point_temperature(°c)  solar_radiation_(mj/m2)  \
0              2000                      -17.6                      0.0   
1              2000                      -17.6                      0.0   
2              2000                      -17.7                      0.0   
3              2000                      -17.6                      0.0   
4              2000                      -18.6                      0.0   

   rainfall(mm)  snowfall_(cm) seasons     holiday functioning_day  

# Exploratory Data Analysis (EDA)

In [4]:
print("\n--- Información General y Tipos de Datos ---")
df.info()

print("\n--- Estadísticas Descriptivas de Variables Numéricas ---")
print(df.describe().T)

print("\n--- Conteo de Valores Únicos en Variables Categóricas/Discretas ---")
print(df['seasons'].value_counts())
print(df['holiday'].value_counts())
print(df['functioning_day'].value_counts())


--- Información General y Tipos de Datos ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date                       8760 non-null   object 
 1   demanda                    8760 non-null   int64  
 2   hour                       8760 non-null   int64  
 3   temperature(°c)            8760 non-null   float64
 4   humidity(%)                8760 non-null   int64  
 5   wind_speed_(m/s)           8760 non-null   float64
 6   visibility_(10m)           8760 non-null   int64  
 7   dew_point_temperature(°c)  8760 non-null   float64
 8   solar_radiation_(mj/m2)    8760 non-null   float64
 9   rainfall(mm)               8760 non-null   float64
 10  snowfall_(cm)              8760 non-null   float64
 11  seasons                    8760 non-null   object 
 12  holiday                    8760 non-null   object 
 13  fu

# Limpieza y Transformación de Datos

In [5]:
# Convertir 'date' al tipo datetime
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# Combinar fecha y hora en una sola columna para análisis de series de tiempo
df['datetime'] = df['date'] + pd.to_timedelta(df['hour'], unit='h')
df.set_index('datetime', inplace=True)
df.drop(['date', 'hour'], axis=1, inplace=True)

print("\n--- DataFrame después de la conversión de tipos ---")
print(df.head())


--- DataFrame después de la conversión de tipos ---
                     demanda  temperature(°c)  humidity(%)  wind_speed_(m/s)  \
datetime                                                                       
2017-12-01 00:00:00      254             -5.2           37               2.2   
2017-12-01 01:00:00      204             -5.5           38               0.8   
2017-12-01 02:00:00      173             -6.0           39               1.0   
2017-12-01 03:00:00      107             -6.2           40               0.9   
2017-12-01 04:00:00       78             -6.0           36               2.3   

                     visibility_(10m)  dew_point_temperature(°c)  \
datetime                                                           
2017-12-01 00:00:00              2000                      -17.6   
2017-12-01 01:00:00              2000                      -17.6   
2017-12-01 02:00:00              2000                      -17.7   
2017-12-01 03:00:00              2000         

In [6]:
# 1. Verificar Nulos (Métricas)
print("\n--- Conteo de Valores Nulos por Columna ---")
print(df.isnull().sum())
# Si aparecen nulos, la acción recomendada (imputación con media/mediana o eliminación) depende de la cantidad.

# 2. Manejo de Inconsistencias
# Las variables 'functioning_day' deben ser binarias (Yes/No o Fun/NoFunc)
# En este dataset son 'Yes' y 'No', lo cual es consistente.

# Convertir 'functioning_day' y 'holiday' a numérico (0 y 1)
df['functioning_day'] = df['functioning_day'].map({'Yes': 1, 'No': 0})
df['holiday'] = df['holiday'].map({'No Holiday': 0, 'Holiday': 1})


--- Conteo de Valores Nulos por Columna ---
demanda                      0
temperature(°c)              0
humidity(%)                  0
wind_speed_(m/s)             0
visibility_(10m)             0
dew_point_temperature(°c)    0
solar_radiation_(mj/m2)      0
rainfall(mm)                 0
snowfall_(cm)                0
seasons                      0
holiday                      0
functioning_day              0
dtype: int64


In [7]:
# Aplicar la detección de outliers en la variable 'demanda'
Q1 = df['demanda'].quantile(0.25)
Q3 = df['demanda'].quantile(0.75)
IQR = Q3 - Q1

limite_superior = Q3 + 1.5 * IQR

# 1. Conteo de Outliers
outliers_count = df[df['demanda'] > limite_superior].shape[0]
print(f"\n--- Detección de Outliers en Demanda ---")
print(f"Límite Superior (1.5*IQR): {limite_superior:.0f}")
print(f"Número de outliers detectados: {outliers_count}")

# 2. Opción de Limpieza (Imputación por tope o 'Capping')
# En lugar de eliminar, reemplazamos los outliers con el límite superior (Capping).
# Esto es común en modelos de regresión para mantener el tamaño del dataset.
df['demanda_limpia'] = np.where(
    df['demanda'] > limite_superior,
    limite_superior,
    df['demanda']
)

# Comparación del máximo antes y después
print(f"Máximo de Demanda (Original): {df['demanda'].max()}")
print(f"Máximo de Demanda (Limpia): {df['demanda_limpia'].max():.0f}")

# Se puede eliminar la columna original 'demanda' si solo se usará la 'demanda_limpia'
df.drop('demanda', axis=1, inplace=True)
df.rename(columns={'demanda_limpia': 'demanda'}, inplace=True)


--- Detección de Outliers en Demanda ---
Límite Superior (1.5*IQR): 2377
Número de outliers detectados: 158
Máximo de Demanda (Original): 3556
Máximo de Demanda (Limpia): 2377


# Transformaciones Finales (Feature Engineering)

In [8]:
# Aplicar One-Hot Encoding a la variable 'seasons'
df_encoded = pd.get_dummies(df, columns=['seasons'], prefix='season')

print("\n--- DataFrame después del One-Hot Encoding de 'seasons' ---")
print(df_encoded.head())


--- DataFrame después del One-Hot Encoding de 'seasons' ---
                     temperature(°c)  humidity(%)  wind_speed_(m/s)  \
datetime                                                              
2017-12-01 00:00:00             -5.2           37               2.2   
2017-12-01 01:00:00             -5.5           38               0.8   
2017-12-01 02:00:00             -6.0           39               1.0   
2017-12-01 03:00:00             -6.2           40               0.9   
2017-12-01 04:00:00             -6.0           36               2.3   

                     visibility_(10m)  dew_point_temperature(°c)  \
datetime                                                           
2017-12-01 00:00:00              2000                      -17.6   
2017-12-01 01:00:00              2000                      -17.6   
2017-12-01 02:00:00              2000                      -17.7   
2017-12-01 03:00:00              2000                      -17.6   
2017-12-01 04:00:00              

In [9]:
# Crear variables adicionales de tiempo
df_encoded['month'] = df_encoded.index.month
df_encoded['day_of_week'] = df_encoded.index.day_of_week # 0=Lunes, 6=Domingo
df_encoded['is_weekend'] = df_encoded['day_of_week'].apply(lambda x: 1 if x >= 5 else 0) # Sábado y Domingo

print("\n--- DataFrame después del Feature Engineering ---")
print(df_encoded.head())


--- DataFrame después del Feature Engineering ---
                     temperature(°c)  humidity(%)  wind_speed_(m/s)  \
datetime                                                              
2017-12-01 00:00:00             -5.2           37               2.2   
2017-12-01 01:00:00             -5.5           38               0.8   
2017-12-01 02:00:00             -6.0           39               1.0   
2017-12-01 03:00:00             -6.2           40               0.9   
2017-12-01 04:00:00             -6.0           36               2.3   

                     visibility_(10m)  dew_point_temperature(°c)  \
datetime                                                           
2017-12-01 00:00:00              2000                      -17.6   
2017-12-01 01:00:00              2000                      -17.6   
2017-12-01 02:00:00              2000                      -17.7   
2017-12-01 03:00:00              2000                      -17.6   
2017-12-01 04:00:00              2000      